In [4]:
class Cilj:
    def __init__(self, d, r, p):
        self.d = d
        self.r = r
        self.p = p

    def pozicija(self, t, v):
        return self.p + (t - self.r) * v * self.d
    
    def __hash__(self):
        return hash((self.d, self.r, self.p))
    
    def __eq__(self, other):
        return (self.d, self.r, self.p) == (other.d, other.r, other.p)
    
    def __repr__(self):
        return f"Cilj({self.d}, {self.r}, {self.p})"

In [18]:
class Ureditev:
    def __init__(self, seznam, kljuc):
        self.seznam = sorted(seznam, key=kljuc)
        self.indeksi = {c: i for i, c in enumerate(self.seznam)}

    def __len__(self):
        return len(self.seznam)
        
    def __getitem__(self, index):
        if index < 0:
            return None
        return self.seznam[index]
    
    def indeks(self, cilj):
        return self.indeksi[cilj]
    
class ObratnaUreditev:
    def __init__(self, ureditev):
        self.ureditev = ureditev
        
    def __len__(self):
        return len(self.ureditev)
        
    def __getitem__(self, index):
        if index < 0:
            return None
        return self.ureditev.seznam[-index-1]
    
    def indeks(self, cilj):
        return len(self) - self.ureditev.indeks(cilj) - 1

In [19]:
cilji = [Cilj(*p) for p in zip([1, 1, -1, -1, -1, 1, -1], [0, 5, 13, 15, 6, 8, 18], [2, 5, -3, 4, 1, -2, -7])]

In [20]:
v = 0.3
IPO = Ureditev(cilji, lambda c: c.pozicija(0, v))
TO = Ureditev(cilji, lambda c: (c.d, c.pozicija(0, v)))
IPOc = ObratnaUreditev(IPO)
TOc = ObratnaUreditev(TO)

In [15]:
[IPOc.indeks(c) for c in cilji]

[3, 1, 4, 0, 2, 6, 5]

In [12]:
cilji

[Cilj(1, 0, 2),
 Cilj(1, 5, 5),
 Cilj(-1, 13, -3),
 Cilj(-1, 15, 4),
 Cilj(-1, 6, 1),
 Cilj(1, 8, -2),
 Cilj(-1, 18, -7)]

In [21]:
IPOc[2]

Cilj(-1, 6, 1)

In [ ]:
class SLMTTSP:
    def __init__(self, cilji, v):
        IPO = Ureditev(cilji, lambda c: c.pozicija(0, v))
        TO = Ureditev(cilji, lambda c: (c.d, c.pozicija(0, v)))
        IPOc = ObratnaUreditev(IPO)
        TOc = ObratnaUreditev(TO)
        self.v = v
        self.cilji = [None] + cilji
        self.ureditve = [IPO, IPOc, TO, TOc]
        self.F = {}

    def g(self, t, j, i):
        posi = i.pozicija(t, self.v)
        posj = j.pozicija(t, self.v)
        razlika = posi - posj
        delta = 1 if razlika > 0 else -1
        tt = t + razlika / (delta - self.v * i.d)
        if tt <= i.r:
            return (tt, [(t, posj), (tt, posi)])
        else:
            return (i.r, [(t, posj), ("čas, ko dosežemo i.p", i.p), (i.r, i.p)])
        
    def predhodnik(self, l, C, i):
        ...

    def predhodno_stanje(self, C, i):
        ...
        
    def f(self, C, i):
        if (C, i) not in F:
            if C == (-1, -1, -1, -1):
                return 0
            kandidati = []
            CC = self.predhodno_stanje(C, i)
            for l in range(4):
                j = self.predhodnik(l, C, i)
                t, _, _ = self.f(CC, j)
                kandidati.append((self.g(t, j, i), l, j))
            self.F[C, i] = min(kandidati)
        return self.F[C, i]
    
    def resi(self):
        n = len(self.cilji)
        for a in range(-1, n+1):
            for b in range(-1, n+1):
                for c in range(-1, n+1):
                    for d in range(-1, n+1):
                        for i in cilji:
                            self.f((a, b, c, d), i)
        return min(self.f((n, n, n, n), i) for i in cilji)
    
    def rekonstruiraj_resitev(self):
        ...